In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dataset = pd.read_csv('E:\ELL_project\problem1\health_data.csv')
dataset = dataset.sample(frac = 1)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values
datasize = X.shape[0]
X_train = X[:(datasize*7)//10,:]
y_train = y[:(datasize*7)//10,:]
X_test = X[(datasize*7)//10:,:]
y_test = y[(datasize*7)//10:,:]

In [ ]:
def gradDesc(X,y,theta):
    h  = np.dot(X,theta)
    dsc = X.shape[0]
    loss = h-y
    total_loss = np.dot(np.ones((1,dsc)),np.square(loss))[0,0] /dsc
    update = np.dot(np.transpose(X),loss)
    update /= dsc
    return(total_loss,update)

In [ ]:
def linReg(X,y,iter=100,alpha=0.01,batchSize=32):
    theta = np.random.random((X.shape[1],1))
    # print(theta)
    datasize = X.shape[0]
    loss_epoch = 0
    for i in range(iter):
        if((i+1)%1000==0 and i>0):
            print('Loss for {} iterations: {}'.format(i+1,loss_epoch))
        fro = 0
        loss_epoch = 0
        while(True):
            to = min(fro+batchSize,datasize)
            l,theta_grad = gradDesc(X[fro:to,:],y[fro:to,:],theta)
            # print(theta_grad)
            loss_epoch += l
            theta -= (alpha*theta_grad)
            fro = to

            if(to>=datasize):
                break
    
    return (theta,loss_epoch)


In [ ]:
train_datasize = X_train.shape[0]
opt_theta, train_loss = linReg(X_train,y_train,20000,0.00001,train_datasize)
print(opt_theta)

Loss for 1000 iterations: 1.4748575204208234
Loss for 2000 iterations: 0.3437451824575775
Loss for 3000 iterations: 0.1531795260161867
Loss for 4000 iterations: 0.12105176629054017
Loss for 5000 iterations: 0.11563529545160073
Loss for 6000 iterations: 0.11472212389512414
Loss for 7000 iterations: 0.11456817082930808
Loss for 8000 iterations: 0.11454221563358824
Loss for 9000 iterations: 0.11453783980527144
Loss for 10000 iterations: 0.1145371020773165
Loss for 11000 iterations: 0.1145369777025705
Loss for 12000 iterations: 0.11453695673403141
Loss for 13000 iterations: 0.11453695319891156
Loss for 14000 iterations: 0.11453695260292002
Loss for 15000 iterations: 0.11453695250244086
Loss for 16000 iterations: 0.1145369524855009
Loss for 17000 iterations: 0.11453695248264498
Loss for 18000 iterations: 0.1145369524821635
Loss for 19000 iterations: 0.11453695248208234
Loss for 20000 iterations: 0.11453695248206865
[[ 0.03024733]
 [-0.0049852 ]
 [-0.00071973]]


In [ ]:
y_pred = np.dot(X_test,opt_theta)
loss_y = y_pred - y_test

test_size = y_pred.shape[0]
total_loss_y = np.dot(np.ones((1,test_size)),np.square(loss_y))[0,0] / test_size

# print(total_loss_y)

y_pred_thresh = y_pred>=0.5

tp = np.sum((y_pred_thresh+y_test)==2 , axis=0)[0]
tn = np.sum(y_pred_thresh==y_test , axis=0)[0] - tp
fp = np.sum(y_pred_thresh , axis=0)[0]-tp
fn = test_size-tp-tn-fp


print('tp: {} , tn: {} , fp: {} , fn: {}'.format(tp,tn,fp,fn))

acc = (tp+tn)/test_size
prec = (tp)/(tp+fp)
recl = (tp)/(tp+fn)
f1 = 2*prec*recl/(prec+recl)

print('Accuracy: {}'.format( acc  ))
print('Precision: {}'.format( prec  ))
print('Recall: {}'.format( recl  ))
print('F1 score: {}'.format( f1  ))




tp: 77 , tn: 98 , fp: 20 , fn: 15
Accuracy: 0.8333333333333334
Precision: 0.7938144329896907
Recall: 0.8369565217391305
F1 score: 0.8148148148148148
